<a href="https://colab.research.google.com/github/etarazonav/650044-ABD-ULIMA/blob/main/Notebooks/ABD_MLlib_Clustering_kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Universidad_de_Lima_logo.png/220px-Universidad_de_Lima_logo.png"  width="120" />  MLlib: Clustering (Agrupamiento)
**Profesor:** Enver G. Tarazona Vargas <br>
**Curso:** Analítica con Big Data <br>
**FACULTAD DE INGENIERÍA - CARRERA DE INGENIERÍA DE SISTEMAS**<br>

# Clustering (Agrupamiento) usando K-means


La implementación de MLlib incluye una versión paralelizada del método <a href="http://en.wikipedia.org/wiki/K-means%2B%2B">k-means++</a>, llamada <a href="http://theory.stanford.edu/~sergei/papers/vldb12-kmpar.pdf">kmeans||</a>.

Los datos de este ejemplo están adaptados del repositorio UCI: https://archive.ics.uci.edu/ml/datasets/seeds.

Los datos contienen datos de granos de trigo correspondientes a tres variedades diferentes. Cada variedad tiene 70 elementos seleccionados aleatoriamente. Se utilizó rayos X para realizar la visualización de la estructura interna del grano.

In [ ]:
# Solo necesario si se usa Google Colab
!pip install -q pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans

spark = SparkSession.builder.getOrCreate()

In [ ]:
# Carga de archivos
!wget -q https://raw.githubusercontent.com/etarazonav/650044-ABD-ULIMA/refs/heads/main/Datos/datos_semillas.csv

In [ ]:
# Cargar los datos
dfsemillas = spark.read.csv("datos_semillas.csv", header=True, inferSchema=True)

#dfsemillas.printSchema()
dfsemillas.show(5)

In [ ]:
# Resumen de los datos
dfsemillas.describe().show()

## 1.&nbsp;Pre-procesamiento de datos

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
# Preparar los datos en una columna llamada "atributos"
vec_assembler = VectorAssembler(inputCols=dfsemillas.columns,
                                outputCol='atributos')

# Transformar los datos
df0 = vec_assembler.transform(dfsemillas)

# Mostrar el DF: se añade una columna al final con los "atributos" en formato adecuado para Spark
df0.show(5)

Al realizar agrupamiento solo se tiene los atributos (sin etiquetas), por lo que aquí se quitará el resto y solo se utilizará la columna de atributos llamada, en este caso, `atributos`

In [ ]:
df1 = df0.select('atributos')
df1.show(5, truncate=False)

Debido a que se utilizará k-means, y este método se basa en distancias, se realizará el escalamiento de los datos

In [ ]:
from pyspark.ml.feature import StandardScaler

In [ ]:
# El valor de salida es z = (x-media)/(deviacion_estandar)
scaler = StandardScaler(inputCol="atributos", outputCol="atributos_escalados",
                        withStd=True, withMean=True)

# Transformar los datos (estandarizar o normalizar)
df = scaler.fit(df1).transform(df1)
df.show(5, truncate=False)

### Entrenamiento

In [ ]:
# Creación del modelo: usando 3 clústeres
kmeans = KMeans(featuresCol='atributos_escalados',
                predictionCol='prediccion',
                k=3)

# Entrenar el modelo
modelo = kmeans.fit(df)

In [ ]:
# Evaluar a través del SSE dentro de la clase
sse = modelo.summary.trainingCost
print("Suma de errores cuadráticos dentro de la clase: {:.3f}".format(sse))

In [ ]:
# Centros de los clústeres
centros = modelo.clusterCenters()

print("Centros de los clústeres: ")
for centro in centros:
    print(centro)

Luego se intentará ver el error con diferentes valores de K. En este caso no es necesario ya que a priori se sabe que hay solamente 3 grupos, pero se incluye aquí solo para ilustrar cómo se podría realizar.

In [ ]:
# Escoger el número de clústeres
import numpy as np
import matplotlib.pyplot as plt

Ks = np.arange(2, 26)
SSEs = np.zeros(Ks.shape)

for i, k in enumerate(Ks):
    kmeans2 = KMeans(featuresCol='atributos_escalados', k=k)
    modelo2 = kmeans2.fit(df)
    sse = modelo2.summary.trainingCost
    SSEs[i] = sse

plt.plot(Ks, SSEs)
plt.xlabel('k'); plt.ylabel('SSE')
plt.grid()
plt.show()

In [ ]:
plt.plot(Ks, SSEs)
plt.plot(Ks, SSEs, 'r.')
plt.xlabel('k'); plt.ylabel('SSE')
plt.grid()
plt.show()

In [ ]:
n1 = len(SSEs)-1
dif = np.zeros(n1)
for i in range(n1):
  dif[i] = np.abs(SSEs[i+1] - SSEs[i])
plt.plot(dif,'.')
plt.grid()

### Predicción

In [ ]:
# Predicción de las clases
prediccion = modelo.transform(df)

# Esquema (se agrega "prediction" al final del data frame)
prediccion.printSchema()

In [ ]:
# Seleccionar solamente los atributos y las predicciones
prediccion.select(['atributos_escalados', 'prediccion']).show(5, truncate=False)